In [ ]:
# Clone the YOLOv5 repository and install dependencies
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 16680, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 16680 (delta 101), reused 129 (delta 63), pack-reused 16459
Receiving objects: 100% (16680/16680), 15.39 MiB | 14.43 MiB/s, done.
Resolving deltas: 100% (11404/11404), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.6/793.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3

In [ ]:
!pwd

/content/yolov5


In [ ]:
import torch
import cv2
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from google.colab import files
import os

In [ ]:
# !pip install --upgrade pillow

In [ ]:
# # Upload the image file
# uploaded = files.upload()

# # Verify the uploaded file
# for filename in uploaded.keys():
#     print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

In [ ]:
# List files in the directory to verify the image file is present
image_directory = '/content/yolov5'
files_in_directory = os.listdir(image_directory)
print("Files in the directory:", files_in_directory)

# Check file extension
for file in files_in_directory:
    if file.endswith(('.jpg', '.jpeg', '.png')):
        print(f"Valid image file found: {file}")
    else:
        print(f"Invalid file format: {file}")

Files in the directory: ['benchmarks.py', 'val.py', 'segment', 'CONTRIBUTING.md', 'utils', '.gitattributes', 'classify', 'requirements.txt', '.gitignore', 'train.py', '.github', '.dockerignore', 'models', 'pyproject.toml', 'best.pt', 'README.md', '.git', 'hubconf.py', 'detect.py', 'data', 'CITATION.cff', 'LICENSE', 'export.py', 'IMG_20230102_140213.jpg', 'tutorial.ipynb', 'README.zh-CN.md']
Invalid file format: benchmarks.py
Invalid file format: val.py
Invalid file format: segment
Invalid file format: CONTRIBUTING.md
Invalid file format: utils
Invalid file format: .gitattributes
Invalid file format: classify
Invalid file format: requirements.txt
Invalid file format: .gitignore
Invalid file format: train.py
Invalid file format: .github
Invalid file format: .dockerignore
Invalid file format: models
Invalid file format: pyproject.toml
Invalid file format: best.pt
Invalid file format: README.md
Invalid file format: .git
Invalid file format: hubconf.py
Invalid file format: detect.py
Invalid

In [ ]:
# Verify the image file path
image_path = '/content/yolov5/IMG_20230102_140213.jpg'  # Replace with the actual file name

if os.path.isfile(image_path):
    print(f"File {image_path} exists.")
else:
    print(f"File {image_path} does not exist.")

# Read the image using OpenCV
image_bgr = cv2.imread(image_path)

if image_bgr is not None:
    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Convert OpenCV image to PIL format
    image_pil = Image.fromarray(image_rgb)

    # Display the image to verify it is loaded correctly
    image_pil.show()

    # Preprocess the image
    transform = transforms.Compose([transforms.Resize((640, 640)),
                                    transforms.ToTensor()])
    image = transform(image_pil).unsqueeze(0)  # Add batch dimension

    # Load custom-trained YOLOv5 model
    custom_weights_path = '/content/yolov5/best.pt'
    try:
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=custom_weights_path, force_reload=True)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Error loading model: {e}")

    # Register hooks to all convolutional layers
    feature_maps = []

    def hook_fn(module, input, output):
        print(f"Hook registered on {module.__class__.__name__}")
        feature_maps.append(output)

    # Hook all convolutional layers within the model
    for name, layer in model.model.named_modules():
        if isinstance(layer, torch.nn.Conv2d):
            layer.register_forward_hook(hook_fn)

    # Run the image through the model
    try:
        with torch.no_grad():
            model(image)
        print(f"Number of feature maps collected: {len(feature_maps)}")
    except Exception as e:
        print(f"Error running the model: {e}")

    # Function to plot and save feature maps
    def plot_and_save_feature_maps(feature_maps, num_cols=8):
        for idx, fmap in enumerate(feature_maps):
            num_features = fmap.size(1)
            num_rows = (num_features + num_cols - 1) // num_cols
            fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 15))
            fig.suptitle(f'Feature Map {idx + 1}', fontsize=16)
            for i in range(num_features):
                ax = axes[i // num_cols, i % num_cols]
                ax.imshow(fmap[0, i].cpu().numpy(), cmap='viridis')
                ax.axis('off')
            plt.tight_layout()
            save_path = f'feature_map_{idx + 1}.png'
            plt.savefig(save_path, format='png')
            plt.close(fig)
            print(f'Saved {save_path}')
            files.download(save_path)

    # Plot and save the feature maps
    if feature_maps:
        plot_and_save_feature_maps(feature_maps)
    else:
        print("No feature maps collected.")
else:
    print("Error: The image could not be read using OpenCV.")

File /content/yolov5/IMG_20230102_140213.jpg exists.


/usr/local/lib/python3.10/dist-packages/torch/hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-6-28 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5m summary: 212 layers, 20881221 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


Model loaded successfully.
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook registered on Conv2d
Hook regist

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [ ]:
# Verify the image file path
image_path = '/content/yolov5/IMG_20230102_140213.jpg'  # Replace with the actual file name

if os.path.isfile(image_path):
    print(f"File {image_path} exists.")
else:
    print(f"File {image_path} does not exist.")

# Read the image using OpenCV
image_bgr = cv2.imread(image_path)

if image_bgr is not None:
    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Convert OpenCV image to PIL format
    image_pil = Image.fromarray(image_rgb)

    # Display the image to verify it is loaded correctly
    image_pil.show()

    # Preprocess the image
    transform = transforms.Compose([transforms.Resize((640, 640)),
                                    transforms.ToTensor()])
    image = transform(image_pil).unsqueeze(0)  # Add batch dimension

    # Load custom-trained YOLOv5 model
    custom_weights_path = '/content/yolov5/best.pt'
    try:
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=custom_weights_path, force_reload=True)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Error loading model: {e}")

    # Register hooks to all convolutional layers
    feature_maps = []

    def hook_fn(module, input, output):
        feature_maps.append(output)

    # Hook all convolutional layers within the model
    for name, layer in model.model.named_modules():
        if isinstance(layer, torch.nn.Conv2d):
            layer.register_forward_hook(hook_fn)

    # Run the image through the model
    try:
        with torch.no_grad():
            model(image)
        print(f"Number of feature maps collected: {len(feature_maps)}")
    except Exception as e:
        print(f"Error running the model: {e}")

    # Function to plot and save feature maps
    def plot_and_save_feature_maps(feature_maps, start_idx, end_idx, num_cols=8):
        for idx in range(start_idx, end_idx):
            if idx >= len(feature_maps):
                break
            fmap = feature_maps[idx]
            num_features = fmap.size(1)
            num_rows = (num_features + num_cols - 1) // num_cols
            fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 15))
            fig.suptitle(f'Feature Map {idx + 1}', fontsize=16)
            for i in range(num_features):
                ax = axes[i // num_cols, i % num_cols]
                ax.imshow(fmap[0, i].cpu().numpy(), cmap='viridis')
                ax.axis('off')
            plt.tight_layout()
            save_path = f'feature_map_{idx + 1}.png'
            plt.savefig(save_path, format='png')
            plt.close(fig)
            print(f'Saved {save_path}')
            files.download(save_path)

    # Plot and save feature maps from 1 to 35
    if feature_maps:
        plot_and_save_feature_maps(feature_maps, 0, 19)
    else:
        print("No feature maps collected.")
else:
    print("Error: The image could not be read using OpenCV.")

File /content/yolov5/IMG_20230102_140213.jpg exists.


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-6-28 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5m summary: 212 layers, 20881221 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


Model loaded successfully.
Number of feature maps collected: 82
Saved feature_map_1.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved feature_map_19.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Verify the image file path
image_path = '/content/yolov5/IMG_20230102_140213.jpg'  # Replace with the actual file name

if os.path.isfile(image_path):
    print(f"File {image_path} exists.")
else:
    print(f"File {image_path} does not exist.")

# Read the image using OpenCV
image_bgr = cv2.imread(image_path)

if image_bgr is not None:
    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Convert OpenCV image to PIL format
    image_pil = Image.fromarray(image_rgb)

    # Display the image to verify it is loaded correctly
    image_pil.show()

    # Preprocess the image
    transform = transforms.Compose([transforms.Resize((640, 640)),
                                    transforms.ToTensor()])
    image = transform(image_pil).unsqueeze(0)  # Add batch dimension

    # Load custom-trained YOLOv5 model
    custom_weights_path = '/content/yolov5/best.pt'
    try:
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=custom_weights_path, force_reload=True)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Error loading model: {e}")

    # Register hooks to all convolutional layers
    feature_maps = []

    def hook_fn(module, input, output):
        feature_maps.append(output)

    # Hook all convolutional layers within the model
    for name, layer in model.model.named_modules():
        if isinstance(layer, torch.nn.Conv2d):
            layer.register_forward_hook(hook_fn)

    # Run the image through the model
    try:
        with torch.no_grad():
            model(image)
        print(f"Number of feature maps collected: {len(feature_maps)}")
    except Exception as e:
        print(f"Error running the model: {e}")

    # Function to plot and save feature maps
    def plot_and_save_feature_maps(feature_maps, start_idx, end_idx, num_cols=8):
        for idx in range(start_idx, end_idx):
            if idx >= len(feature_maps):
                break
            fmap = feature_maps[idx]
            num_features = fmap.size(1)
            num_rows = (num_features + num_cols - 1) // num_cols
            fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 15))
            fig.suptitle(f'Feature Map {idx + 1}', fontsize=16)
            for i in range(num_features):
                ax = axes[i // num_cols, i % num_cols]
                ax.imshow(fmap[0, i].cpu().numpy(), cmap='viridis')
                ax.axis('off')
            plt.tight_layout()
            save_path = f'feature_map_{idx + 1}.png'
            plt.savefig(save_path, format='png')
            plt.close(fig)
            print(f'Saved {save_path}')
            files.download(save_path)

    # Plot and save feature maps from 36 to 70
    if feature_maps:
        plot_and_save_feature_maps(feature_maps, 20, 39)
    else:
        print("No feature maps collected.")
else:
    print("Error: The image could not be read using OpenCV.")

File /content/yolov5/IMG_20230102_140213.jpg exists.


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-6-28 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5m summary: 212 layers, 20881221 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


Model loaded successfully.
Number of feature maps collected: 82
Saved feature_map_21.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved feature_map_39.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Verify the image file path
image_path = '/content/yolov5/IMG_20230102_140213.jpg'  # Replace with the actual file name

if os.path.isfile(image_path):
    print(f"File {image_path} exists.")
else:
    print(f"File {image_path} does not exist.")

# Read the image using OpenCV
image_bgr = cv2.imread(image_path)

if image_bgr is not None:
    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Convert OpenCV image to PIL format
    image_pil = Image.fromarray(image_rgb)

    # Display the image to verify it is loaded correctly
    image_pil.show()

    # Preprocess the image
    transform = transforms.Compose([transforms.Resize((640, 640)),
                                    transforms.ToTensor()])
    image = transform(image_pil).unsqueeze(0)  # Add batch dimension

    # Load custom-trained YOLOv5 model
    custom_weights_path = '/content/yolov5/best.pt'
    try:
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=custom_weights_path, force_reload=True)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Error loading model: {e}")

    # Register hooks to all convolutional layers
    feature_maps = []

    def hook_fn(module, input, output):
        feature_maps.append(output)

    # Hook all convolutional layers within the model
    for name, layer in model.model.named_modules():
        if isinstance(layer, torch.nn.Conv2d):
            layer.register_forward_hook(hook_fn)

    # Run the image through the model
    try:
        with torch.no_grad():
            model(image)
        print(f"Number of feature maps collected: {len(feature_maps)}")
    except Exception as e:
        print(f"Error running the model: {e}")

    # Function to plot and save feature maps
    def plot_and_save_feature_maps(feature_maps, start_idx, end_idx, num_cols=8):
        for idx in range(start_idx, end_idx):
            if idx >= len(feature_maps):
                break
            fmap = feature_maps[idx]
            num_features = fmap.size(1)
            num_rows = (num_features + num_cols - 1) // num_cols
            fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 15))
            fig.suptitle(f'Feature Map {idx + 1}', fontsize=16)
            for i in range(num_features):
                ax = axes[i // num_cols, i % num_cols]
                ax.imshow(fmap[0, i].cpu().numpy(), cmap='viridis')
                ax.axis('off')
            plt.tight_layout()
            save_path = f'feature_map_{idx + 1}.png'
            plt.savefig(save_path, format='png')
            plt.close(fig)
            print(f'Saved {save_path}')
            files.download(save_path)

    # Plot and save feature maps from 36 to 70
    if feature_maps:
        plot_and_save_feature_maps(feature_maps, 40, 59)
    else:
        print("No feature maps collected.")
else:
    print("Error: The image could not be read using OpenCV.")

File /content/yolov5/IMG_20230102_140213.jpg exists.


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-6-28 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5m summary: 212 layers, 20881221 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


Model loaded successfully.
Number of feature maps collected: 82
Saved feature_map_41.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved feature_map_59.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Verify the image file path
image_path = '/content/yolov5/IMG_20230102_140213.jpg'  # Replace with the actual file name

if os.path.isfile(image_path):
    print(f"File {image_path} exists.")
else:
    print(f"File {image_path} does not exist.")

# Read the image using OpenCV
image_bgr = cv2.imread(image_path)

if image_bgr is not None:
    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Convert OpenCV image to PIL format
    image_pil = Image.fromarray(image_rgb)

    # Display the image to verify it is loaded correctly
    image_pil.show()

    # Preprocess the image
    transform = transforms.Compose([transforms.Resize((640, 640)),
                                    transforms.ToTensor()])
    image = transform(image_pil).unsqueeze(0)  # Add batch dimension

    # Load custom-trained YOLOv5 model
    custom_weights_path = '/content/yolov5/best.pt'
    try:
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=custom_weights_path, force_reload=True)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Error loading model: {e}")

    # Register hooks to all convolutional layers
    feature_maps = []

    def hook_fn(module, input, output):
        feature_maps.append(output)

    # Hook all convolutional layers within the model
    for name, layer in model.model.named_modules():
        if isinstance(layer, torch.nn.Conv2d):
            layer.register_forward_hook(hook_fn)

    # Run the image through the model
    try:
        with torch.no_grad():
            model(image)
        print(f"Number of feature maps collected: {len(feature_maps)}")
    except Exception as e:
        print(f"Error running the model: {e}")

    # Function to plot and save feature maps
    def plot_and_save_feature_maps(feature_maps, start_idx, end_idx, num_cols=8):
        for idx in range(start_idx, end_idx):
            if idx >= len(feature_maps):
                break
            fmap = feature_maps[idx]
            num_features = fmap.size(1)
            num_rows = (num_features + num_cols - 1) // num_cols
            fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 15))
            fig.suptitle(f'Feature Map {idx + 1}', fontsize=16)
            for i in range(num_features):
                ax = axes[i // num_cols, i % num_cols]
                ax.imshow(fmap[0, i].cpu().numpy(), cmap='viridis')
                ax.axis('off')
            plt.tight_layout()
            save_path = f'feature_map_{idx + 1}.png'
            plt.savefig(save_path, format='png')
            plt.close(fig)
            print(f'Saved {save_path}')
            files.download(save_path)

    # Plot and save feature maps from 36 to 70
    if feature_maps:
        plot_and_save_feature_maps(feature_maps, 60, 82)
    else:
        print("No feature maps collected.")
else:
    print("Error: The image could not be read using OpenCV.")

File /content/yolov5/IMG_20230102_140213.jpg exists.


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-6-28 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5m summary: 212 layers, 20881221 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


Model loaded successfully.
Number of feature maps collected: 82
Saved feature_map_61.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved feature_map_81.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>